# BINARY TEXT CLASSIFICATION USING BERT

Бинарная классификация позитивных и негативных отзывов по IMDb movie review dataset с использованием преодобученного тансформера Bert [BERT](https://arxiv.org/abs/1810.04805). Оберткой над Keras для обучения будем использовать библиотеку ktrain [*ktrain* library](https://github.com/amaiya/ktrain)

Данильченко Вадим

In [0]:
# install ktrain
# !pip3 install ktrain

In [0]:
import ktrain
from ktrain import text

In [0]:
ktrain.__version__

'0.14.1'

In [0]:
# скачаем датасет 
import tensorflow as tf
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 8s 0us/step


In [0]:
# распакуем данные
import os.path
#dataset = '/root/.keras/datasets/aclImdb'
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(IMDB_DATADIR)

/root/.keras/datasets/aclImdb


#### загрузка и препроцессинг датасета

In [0]:
# загрузим скаченные данные и преобразуем в специальный формат, с которым BERT умеет работать
# также будет подгружена модель и  словарь
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


#### обернем в объект ktrain.Learner предобученную модель BERT

In [0]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


#### подберем лучший learning rate и обучим модель

In [0]:
# найдем оптимальный learning rate
learner.lr_find()
learner.lr_plot()

In [0]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 3411s 136ms/sample - loss: 0.2615 - accuracy: 0.8910 - val_loss: 0.1644 - val_accuracy: 0.9371


#### предскажем класс для нескольких сгенерированных фраз

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
predictor.get_classes()

['neg', 'pos']

In [0]:
data = ["i don't like this", 
        "it was terrible",
        "what an idiot made it",
        "i've never seen something better",
        "what a beautifull movie",
        "yes i like it"]

In [0]:
predictor.predict(data, return_proba=True)

array([[0.83540404, 0.16459592],
       [0.96164936, 0.03835063],
       [0.93933153, 0.06066843],
       [0.06051107, 0.9394889 ],
       [0.02821554, 0.9717845 ],
       [0.05389027, 0.9461097 ]], dtype=float32)

In [0]:
predictor.save('data/IMDb_predictor')

In [0]:
# predictor = ktrain.load_predictor('/tmp/my_moviereview_predictor')
# predictor.predict(['this is my favorite movie'])